In [2]:
# Import required libraries
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# Load data
df = pd.read_csv("/content/animal_disease_dataset.csv")

# Combine symptoms into a single list of unique symptoms
all_symptoms = pd.Series(df[['Symptom 1', 'Symptom 2', 'Symptom 3']].values.ravel()).unique()

# One-hot encode symptoms
for symptom in all_symptoms:
    df[symptom] = df.apply(lambda row: int(symptom in [row['Symptom 1'], row['Symptom 2'], row['Symptom 3']]), axis=1)

# Drop original symptom columns
df.drop(['Symptom 1', 'Symptom 2', 'Symptom 3'], axis=1, inplace=True)

# Encode 'Animal' and 'Disease' columns
le_animal = LabelEncoder()
df['Animal'] = le_animal.fit_transform(df['Animal'])

le_disease = LabelEncoder()
df['Disease'] = le_disease.fit_transform(df['Disease'])

# Features and target
X = df.drop('Disease', axis=1)
y = df['Disease']

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# KNN Classifier
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)

# Predictions and Evaluation
y_pred = knn.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))


Accuracy: 0.8111009593421654

Confusion Matrix:
 [[1972    0    0    0    0]
 [   0 1960    3    0    0]
 [   0    0 1963    0    0]
 [   0    0    0  579  864]
 [   0    0    0  787  628]]

Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00      1972
           1       1.00      1.00      1.00      1963
           2       1.00      1.00      1.00      1963
           3       0.42      0.40      0.41      1443
           4       0.42      0.44      0.43      1415

    accuracy                           0.81      8756
   macro avg       0.77      0.77      0.77      8756
weighted avg       0.81      0.81      0.81      8756



In [5]:
# Function to predict disease based on user input
def predict_from_user_input():
    print("\n--- Animal Disease Predictor ---")

    # Show valid animal types
    print("\nAvailable Animal Types:", list(le_animal.classes_))
    animal = input("Enter animal type: ").strip().lower()
    if animal not in le_animal.classes_:
        print("Invalid animal type!")
        return

    age = int(input("Enter age of the animal: "))
    temp = float(input("Enter temperature of the animal: "))

    print("\nAvailable Symptoms:", list(all_symptoms))
    symptoms = []
    for i in range(1, 4):
        symptom = input(f"Enter Symptom {i}: ").strip().lower()
        if symptom not in all_symptoms:
            print("Invalid symptom!")
            return
        symptoms.append(symptom)

    # Create input row
    input_data = dict.fromkeys(X.columns, 0)
    input_data['Animal'] = le_animal.transform([animal])[0]
    input_data['Age'] = age
    input_data['Temperature'] = temp

    for symptom in symptoms:
        if symptom in input_data:
            input_data[symptom] = 1

    input_df = pd.DataFrame([input_data])
    prediction_encoded = knn.predict(input_df)[0]
    prediction = le_disease.inverse_transform([prediction_encoded])[0]

    print(f"\n🩺 Predicted Disease: {prediction}")

# Run the input function
predict_from_user_input()



--- Animal Disease Predictor ---

Available Animal Types: ['buffalo', 'cow', 'goat', 'sheep']
Enter animal type: cow
Enter age of the animal: 34
Enter temperature of the animal: 100

Available Symptoms: ['depression', 'painless lumps', 'loss of appetite', 'swelling in limb', 'crackling sound', 'blisters on gums', 'difficulty walking', 'blisters on tongue', 'lameness', 'blisters on mouth', 'chills', 'swelling in extremities', 'shortness of breath', 'sores on mouth', 'sores on tongue', 'sores on gums', 'fatigue', 'sweats', 'chest discomfort', 'sores on hooves', 'swelling in abdomen', 'swelling in muscle', 'blisters on hooves', 'swelling in neck']
Enter Symptom 1: depression
Enter Symptom 2: painless lumps
Enter Symptom 3: blisters on gums

🩺 Predicted Disease: pneumonia
